# find-about-pages-v3
SanjayKAroraPhD@gmail.com <br>
December 2018

## Description
Most of the interesting company information is not on home pages but on about us pages.  This script finds those links from the home pages for further crawling. 

## Change log
v3 treats this is an exercise for identifying inputs to crawling.

## TODO:
* Need to make better use of all pages in the site, e.g., to improve data quality and use additional paragraph data found on non-homepages 
* Standardize firm name cleaning code utility into a universally accessible module
* Port this to real-time crawler? 

In [1]:
# import data processing and other libraries
import csv
import sys
import requests
import os
import re
import pprint
import pymongo
import traceback
from time import sleep
import requests
import pandas as pd
import io
from IPython.display import display
import time
import numpy as np
from bs4 import BeautifulSoup
import string
from urllib.parse import urlparse, urljoin
import requests

In [2]:
from boilerpipe.extract import Extractor

In [7]:
MONGODB_DB = "FirmDB_20181203"
MONGODB_COLLECTION = "pages_depth0"
CONNECTION_STRING = "mongodb://localhost"
# username = "scrapy"
# password = "eager"
# authSource = "FirmDB"
# authMechanism='SCRAM-SHA-1'

client = pymongo.MongoClient(CONNECTION_STRING)
 # , username=username, password=password, authSource=authSource, authMechanism=authMechanism)
db = client[MONGODB_DB]
col = db[MONGODB_COLLECTION]

OUT_DIR = '/Users/sanjay/dev/EAGER/data/orgs/about/'
OUTF = OUT_DIR + 'about_links4.csv'

In [4]:
# gather unique firm_names from mongodb

def get_firm_aggregates ():
    query = [ { "$group": {"_id":"$firm_name" , "number":{"$sum":1}} } ]
    # test for a specific firm
    # query = [ {"$match":{"firm_name": 'MAHLE International GmbH'}}, { "$group": {"_id":"$firm_name" , "number":{"$sum":1}} } ]
    results = col.aggregate(query)

    mongo_dict = {}
    for result in results:
        key = (result['_id'])
        if key:
            mongo_dict[key[0]] = result['number']
    
    return mongo_dict

results_dict = get_firm_aggregates()
firm_names = results_dict.keys()
print (len(firm_names))
pp = pprint.PrettyPrinter()
pp.pprint(firm_names)

1152
dict_keys(['Metabolix', 'Yeda Research and Development Co.', 'Wizard Labs', 'ZENA TECHNOLOGIES', 'Wireless Environment', 'Weatherford Canada Partnership', 'Vorbeck Materials Corporation', 'Wenger Corporation', 'Vizio Inc.', 'VMware', 'VEECO PRECISION SURFACE PROCESSING LLC', 'DuPont Teijin Films U.S. Limited Partnership', 'Vaxiion Therapeutics', 'Carl Zeiss Meditec AG', 'Xerox Corporation', 'Yageo Corporation', '22nd Century Limited', 'Yamaha Corporation', 'WOVN', 'Valicor', 'Wikipad', 'Transitions Optical', 'Kabushikikaisha Toshiba', 'Wine.com', 'Nederlandse Organisatie voor toegepast-natuurwetenschappelijk onderzoek TNO', 'Thorn Bioscience LLC', 'Wave Energy Conversion Corporation of America', 'Verliant Energy', 'Veracode', 'Xintec Inc.', 'TECNIUM', 'United Technologies Corporation', 'Unistrut International Corp.', 'Tata Consultancy Services Limited', 'Tufts Medical Center', 'W&Wsens Devices', 'Casio Computer Co.', 'Crystal Solar Inc.', 'TT Technologies', 'Xyleco', 'Total Market

In [5]:
# find all links that have about, company, and corporate mentions 
# 1. pull down home page
# 2. get a list of all links in Beautiful Soup
# 3. keep only those that match a regex

firm_about_dict = {}
firm_home_dict = {}

# regex to filter in about pages and filter out non-about looking pages, # should add strategy
regex_in = re.compile('(about|aboutus|company|corporate|who.we.are|(^|/)vision|profile|corporate|management|team|history|mission|values|business.model|partner|supplier|universit|strateg|patent|story|intellectual|ipr|licensing)', flags=re.IGNORECASE)
regex_out = re.compile('(order|sale|api|product|company.?home|pdf$)', flags=re.IGNORECASE)
    
def fix_url(url, home_url, domain):
    joined_url = urljoin (home_url, url, allow_fragments=False)
    # needed for company names with embedded urls, e.g., KT Corporation
    found_url = re.findall('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\), ]|(?:%[0-9a-fA-F][0-9a-fA-F]))+', joined_url) 
    if not found_url:
        return ''
    matches = re.split("','", found_url[0]) 
    return matches[0]

def get_domain (url):
    o = urlparse(url)
    domain = o.netloc
    return domain

def get_link_path (url):
    o = urlparse(url)
    path = o.path + o.params + o.query + o.fragment
    return path

def get_about_links_from_home_pages (): 
    for firm_name in firm_names: 
            
        regex = '^' + re.escape(firm_name) + '$'
        results = col.find( {"firm_name": re.compile(regex, re.IGNORECASE)}).sort("depth",pymongo.ASCENDING) 

        result = results[0]
        html = result['html'][0]
        home_url = result['url'][0]
 
        domain = get_domain(home_url).strip('\/')
        
        # print ("Working on " + firm_name + ' at ' + home_url)
        soup = BeautifulSoup(html, 'lxml')
        abouts = {a.get('href'): ' '.join(str(a.string).split()) for a in soup.find_all('a', href=True) 
                  if regex_in.search(get_link_path(a.get('href'))) or regex_in.search(str(a.string))}
        
        abouts_fixed = {fix_url(link, home_url, domain): text for link, text in abouts.items()} 
 
        abouts_clnd = {link: text for link, text in abouts_fixed.items() if re.search(domain, link)}
        abouts_clnd = {link: text for link, text in abouts_clnd.items() 
                       if not regex_out.search(link) and not regex_out.search(text)}
        
        if abouts_clnd:
            firm_about_dict[firm_name] = abouts_clnd

        firm_home_dict[firm_name] = home_url
            
#     pp.pprint (firm_about_dict)
#     pp.pprint (firm_home_dict)

get_about_links_from_home_pages()

In [8]:
# print output to file 
if os.path.isfile(OUTF):
    os.remove(OUTF)

f_out = open(OUTF, 'a')
csv_out = csv.writer(f_out)

# type is either 'about' or 'home' because each will likely need to be processed differently eventually
csv_out.writerow(["firm_name","url","is_about", "descriptor","total_about_pages"]) 

for firm_key, abouts in firm_about_dict.items():
    sorted_links_by_len = sorted(abouts, key=len)
    total_about_pages = len(sorted_links_by_len)
    # pp.pprint(sorted_links_by_len)
    for link in sorted_links_by_len: 
        values = abouts[link]
        descriptor = '|'.join(values.replace('|','').split())
        csv_out.writerow([firm_key, link,1,descriptor,total_about_pages]) 

f_out.flush()
        
for firm_key, home in firm_home_dict.items():
    total_about_pages = 0
    if firm_key in firm_about_dict:
        total_about_pages = len(firm_about_dict[firm_key])
    csv_out.writerow([firm_key, home,0,'',total_about_pages]) 

f_out.flush()
f_out.close()

In [ ]:
# Now go crawl...